In [1]:
"""
2017 A. R. Malipeddi
A simple 2D geometric multigrid solver for the homogeneous Dirichlet Poisson problem on Cartesian grids and unit square. Cell centered 5-point finite difference operator.
"""

import numpy as np
from scipy.sparse.linalg import LinearOperator

def Jacrelax(level,nx,ny,u,f,iters=1,pre=False):
  '''
  under-relaxed Jacobi method smoothing
  '''

  dx=1.0/nx; dy=1.0/ny
  Ax=1.0/dx**2; Ay=1.0/dy**2
  Ap=1.0/(2.0*(Ax+Ay))

  #Dirichlet BC
  u[ 0,:] = -u[ 1,:]
  u[-1,:] = -u[-2,:]
  u[:, 0] = -u[:, 1]
  u[:,-1] = -u[:,-2]

  #if it is a pre-sweep, u is fully zero (on the finest grid depends on BC, always true on coarse grids)
  # we can save some calculation, if doing only one iteration, which is typically the case.
  if(pre and level>1):
    u[1:nx+1,1:ny+1] = -Ap*f[1:nx+1,1:ny+1]
    #Dirichlet BC
    u[ 0,:] = -u[ 1,:]
    u[-1,:] = -u[-2,:]
    u[:, 0] = -u[:, 1]
    u[:,-1] = -u[:,-2]
    iters=iters-1

  for it in range(iters):
    u[1:nx+1,1:ny+1] = Ap*(Ax*(u[2:nx+2,1:ny+1] + u[0:nx,1:ny+1])
                         + Ay*(u[1:nx+1,2:ny+2] + u[1:nx+1,0:ny])
                         - f[1:nx+1,1:ny+1])
    #Dirichlet BC
    u[ 0,:] = -u[ 1,:]
    u[-1,:] = -u[-2,:]
    u[:, 0] = -u[:, 1]
    u[:,-1] = -u[:,-2]

#  if(not pre):
#    return u,None

  res=np.zeros([nx+2,ny+2])
  res[1:nx+1,1:ny+1]=f[1:nx+1,1:ny+1]-(( Ax*(u[2:nx+2,1:ny+1]+u[0:nx,1:ny+1])
                                       + Ay*(u[1:nx+1,2:ny+2]+u[1:nx+1,0:ny])
                                       - 2.0*(Ax+Ay)*u[1:nx+1,1:ny+1]))
  return u,res

def restrict(nx,ny,v):
  '''
  restrict 'v' to the coarser grid
  '''
  v_c=np.zeros([nx+2,ny+2])

#  #vectorized form of
#  for i in range(1,nx+1):
#    for j in range(1,ny+1):
#      v_c[i,j]=0.25*(v[2*i-1,2*j-1]+v[2*i,2*j-1]+v[2*i-1,2*j]+v[2*i,2*j])
 
  v_c[1:nx+1,1:ny+1]=0.25*(v[1:2*nx:2,1:2*ny:2]+v[1:2*nx:2,2:2*ny+1:2]+v[2:2*nx+1:2,1:2*ny:2]+v[2:2*nx+1:2,2:2*ny+1:2])

  return v_c

def prolong(nx,ny,v):
  '''
  interpolate 'v' to the fine grid
  '''
  v_f=np.zeros([2*nx+2,2*ny+2])

#  #vectorized form of
#  for i in range(1,nx+1):
#    for j in range(1,ny+1):
#      v_f[2*i-1,2*j-1] = 0.5625*v[i,j]+0.1875*(v[i-1,j]+v[i,j-1])+0.0625*v[i-1,j-1]
#      v_f[2*i  ,2*j-1] = 0.5625*v[i,j]+0.1875*(v[i+1,j]+v[i,j-1])+0.0625*v[i+1,j-1]
#      v_f[2*i-1,2*j  ] = 0.5625*v[i,j]+0.1875*(v[i-1,j]+v[i,j+1])+0.0625*v[i-1,j+1]
#      v_f[2*i  ,2*j  ] = 0.5625*v[i,j]+0.1875*(v[i+1,j]+v[i,j+1])+0.0625*v[i+1,j+1]

  a=0.5625; b=0.1875; c= 0.0625

  v_f[1:2*nx:2  ,1:2*ny:2  ] = a*v[1:nx+1,1:ny+1]+b*(v[0:nx  ,1:ny+1]+v[1:nx+1,0:ny]  )+c*v[0:nx  ,0:ny  ]
  v_f[2:2*nx+1:2,1:2*ny:2  ] = a*v[1:nx+1,1:ny+1]+b*(v[2:nx+2,1:ny+1]+v[1:nx+1,0:ny]  )+c*v[2:nx+2,0:ny  ]
  v_f[1:2*nx:2  ,2:2*ny+1:2] = a*v[1:nx+1,1:ny+1]+b*(v[0:nx  ,1:ny+1]+v[1:nx+1,2:ny+2])+c*v[0:nx  ,2:ny+2]
  v_f[2:2*nx+1:2,2:2*ny+1:2] = a*v[1:nx+1,1:ny+1]+b*(v[2:nx+2,1:ny+1]+v[1:nx+1,2:ny+2])+c*v[2:nx+2,2:ny+2]

  return v_f

def V_cycle(nx,ny,num_levels,u,f,level=1):
  '''
  V cycle
  '''
  if(level==num_levels):#bottom solve
    u,res=Jacrelax(level,nx,ny,u,f,iters=50,pre=True)
    return u,res

  #Step 1: Relax Au=f on this grid
  u,res=Jacrelax(level,nx,ny,u,f,iters=2,pre=True)

  #Step 2: Restrict residual to coarse grid
  res_c=restrict(nx//2,ny//2,res)

  #Step 3:Solve A e_c=res_c on the coarse grid. (Recursively)
  e_c=np.zeros_like(res_c)
  e_c,res_c=V_cycle(nx//2,ny//2,num_levels,e_c,res_c,level+1)

  #Step 4: Interpolate(prolong) e_c to fine grid and add to u
  u+=prolong(nx//2,ny//2,e_c)
 
  #Step 5: Relax Au=f on this grid
  u,res=Jacrelax(level,nx,ny,u,f,iters=1,pre=False)
  return u,res

In [2]:
"""
This is an example showing how to call the mgd2d solver.
"""
import numpy as np
import time
#from mgd2d import V_cycle

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm

#analytical solution
def Uann(x,y):
    return np.sin(np.pi*x)*np.sin(np.pi*y)
   #return (x**3-x)*(y**3-y)
#RHS corresponding to above
def source(x,y):
    return -2*np.pi**2*np.sin(np.pi*x)*np.sin(np.pi*y)
  #return 6*x*y*(x**2+ y**2 - 2)

#input
max_cycles = 50   #maximum numbera of V cycles
nlevels    = 8    #number of grid levels. 1 means no multigrid, 2 means one coarse grid. etc
NX         = 1*2**(nlevels-1) #Nx and Ny are given as function of grid levels
NY         = 1*2**(nlevels-1) #
tol        = 1e-10      

#the grid has one layer of ghost cells to help apply the boundary conditions
uann=np.zeros([NX+2,NY+2])#analytical solution
u   =np.zeros([NX+2,NY+2])#approximation
f   =np.zeros([NX+2,NY+2])#RHS

#calcualte the RHS and exact solution
DX=1.0/NX
DY=1.0/NY

xc=np.linspace(0.5*DX,1-0.5*DX,NX)
yc=np.linspace(0.5*DY,1-0.5*DY,NY)
XX,YY=np.meshgrid(xc,yc,indexing='ij')

uann[1:NX+1,1:NY+1]=Uann(XX,YY)
f[1:NX+1,1:NY+1]   =source(XX,YY)

print('mgd2d.py solver:')
print('NX:',NX,', NY:',NY,', tol:',tol,'levels: ',nlevels)

#start solving
tb=time.time()

##V cycle
for it in range(1,max_cycles+1):
  u,res=V_cycle(NX,NY,nlevels,u,f)
  rtol=np.max(np.max(np.abs(res)))
  if(rtol<tol):
    break
  error=uann[1:NX+1,1:NY+1]-u[1:NX+1,1:NY+1]
  print('  cycle: ',it,', L_inf(res.)= ',rtol,',L_inf(true error): ',np.max(np.max(np.abs(error))))

print('Elapsed time: ',time.time()-tb,' seconds')

error=uann[1:NX+1,1:NY+1]-u[1:NX+1,1:NY+1]
print('L_inf (true error): ',np.max(np.max(np.abs(error))))

fig = plt.figure()
ax = fig.gca(projection='3d')
surf = ax.plot_surface(XX, YY, u[1:NX+1,1:NY+1],cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
plt.show()


mgd2d.py solver:
NX: 128 , NY: 128 , tol: 1e-10 levels:  8
  cycle:  1 , L_inf(res.)=  3.8842647443947307 ,L_inf(true error):  0.19625458250859473
  cycle:  2 , L_inf(res.)=  0.7643214228577371 ,L_inf(true error):  0.038487609174354454
  cycle:  3 , L_inf(res.)=  0.15037319982218733 ,L_inf(true error):  0.007515098183853541
  cycle:  4 , L_inf(res.)=  0.02957986900435472 ,L_inf(true error):  0.0014348803208950045
  cycle:  5 , L_inf(res.)=  0.005817781201020722 ,L_inf(true error):  0.0002413175849096394
  cycle:  6 , L_inf(res.)=  0.0011440830697928561 ,L_inf(true error):  7.026543456190026e-06
  cycle:  7 , L_inf(res.)=  0.0002249573429971008 ,L_inf(true error):  3.8962170635148397e-05
  cycle:  8 , L_inf(res.)=  4.422715426599666e-05 ,L_inf(true error):  4.798895466118225e-05
  cycle:  9 , L_inf(res.)=  8.694098486472512e-06 ,L_inf(true error):  4.976069743156675e-05
  cycle:  10 , L_inf(res.)=  1.708917242382313e-06 ,L_inf(true error):  5.010844227926192e-05
  cycle:  11 , L_inf(res

TypeError: gca() got an unexpected keyword argument 'projection'

<Figure size 640x480 with 0 Axes>